In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif

In this study, we aim to replicate the work from paper https://doi.org/10.1016/j.ygeno.2020.06.035 that analysed mRNA expression dataset to discover a validated set of genes with differential expression in the cases with spread to adjacent lymph nodes (N1) compared to local type (N0) of prostate cancer (PCa).

In [2]:
# After downloading the data, use your own pathways
file_path = '/home/DAVIDSON/jiqian/Workspace/IS/data2/data_curated_trans.csv'
data = pd.read_csv(file_path)

A TCGA dataset of Prostate Adenocarcinoma (TCGA, PanCancer Atlas) was provided by the paper https://data.mendeley.com/datasets/fdb8f5hjyd/1, which is retrived from cBioPortal according to the paper. However, we failed to find the same source of data.

We inferred that RNA expression values had been standardized against the gene's expression distribution in a reference population and had been reported as log2 values.

CNA data has been reported as +2, +1, 0, −1, or −2. 


In [3]:
data

,PATIENT_ID,PATH_N_STAGE,NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT,PATH_T_STAGE,PRIOR_DX,RADIATION_THERAPY,AGE,M_UBE2Q2P2,M_HMGB1P1,M_LOC155060,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,TCGA-2A-A8VL,2.0,2.0,1.0,2.0,2.0,51.0,-0.0361,NaN,0.3014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TCGA-2A-A8VO,3.0,2.0,2.0,2.0,2.0,57.0,-0.3004,NaN,0.6841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TCGA-2A-A8VT,1.0,2.0,2.0,2.0,1.0,47.0,3.1758,NaN,2.8191,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,TCGA-2A-A8VV,2.0,2.0,1.0,2.0,2.0,52.0,-0.1102,NaN,-0.1719,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TCGA-2A-A8VX,2.0,2.0,2.0,2.0,2.0,70.0,-0.7052,NaN,0.3580,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,TCGA-ZG-A9M4,1.0,2.0,2.0,2.0,2.0,65.0,0.6768,NaN,10.6747,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0
485,TCGA-ZG-A9MC,1.0,2.0,2.0,2.0,1.0,69.0,-0.2252,NaN,-0.0288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
486,TCGA-ZG-A9N3,1.0,2.0,2.0,2.0,2.0,73.0,-0.8350,NaN,-0.5767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
487,TCGA-ZG-A9ND,2.0,2.0,2.0,2.0,2.0,55.0,0.4274,NaN,-0.0456,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [141]:
#remove irrelavant columns
data = data.drop(['PATIENT_ID', 'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT', 'PATH_T_STAGE',
                  'PRIOR_DX', 'RADIATION_THERAPY', 'AGE'], axis=1)

In [142]:
data

,PATH_N_STAGE,M_UBE2Q2P2,M_HMGB1P1,M_LOC155060,M_RNU12-2P,M_SSX9,M_CXORF67,M_EFCAB8,M_SRP14P1,M_LOC391343,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,2.0,-0.0361,NaN,0.3014,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,-0.3004,NaN,0.6841,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,3.1758,NaN,2.8191,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,2.0,-0.1102,NaN,-0.1719,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,-0.7052,NaN,0.3580,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,1.0,0.6768,NaN,10.6747,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0
485,1.0,-0.2252,NaN,-0.0288,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
486,1.0,-0.8350,NaN,-0.5767,NaN,20.2978,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
487,2.0,0.4274,NaN,-0.0456,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


### Create a tabel with mRNA expression of different genes (features) and stage of cancer (labels) 

We know that the samples had been assigned as either N1 or N0 groups. The N1 group included the samples from the patients with PCa 
with the involvement of lymph nodes whereas N0 group included the samples 
from the patients with PCa without the involvement of any lymph nodes. 
The NA samples were removed from the study.

In [143]:
data['PATH_N_STAGE'].value_counts()

2.0    340
1.0     77
3.0     72
Name: PATH_N_STAGE, dtype: int64

According to the paper, there are 347 N0 patients and 77 N1 patients. Therefore, we conclude that 

In [144]:
N1 = data[data['PATH_N_STAGE'] == 1.0]
N0 = data[data['PATH_N_STAGE'] == 2.0]
data = pd.concat([N0,N1])
#note thar with concat, indexing of data was different
data

,PATH_N_STAGE,M_UBE2Q2P2,M_HMGB1P1,M_LOC155060,M_RNU12-2P,M_SSX9,M_CXORF67,M_EFCAB8,M_SRP14P1,M_LOC391343,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,2.0,-0.0361,NaN,0.3014,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,-0.1102,NaN,-0.1719,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,-0.7052,NaN,0.3580,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,1.4828,NaN,-0.0582,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,-0.1719,NaN,0.1259,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,1.0,-0.5763,NaN,-0.5543,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
482,1.0,0.5183,NaN,-0.0826,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
484,1.0,0.6768,NaN,10.6747,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0
485,1.0,-0.2252,NaN,-0.0288,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We wish to select data for mRNA expression, which are columns that starts wtih "M_"

In [145]:
PATH_N_STAGE = data['PATH_N_STAGE']
# select columns with prefix 'M_'as mRNA expression
data = data.filter(like='M_')
data.insert(loc=0, column='PATH_N_STAGE', value=PATH_N_STAGE) 
data

,PATH_N_STAGE,M_UBE2Q2P2,M_HMGB1P1,M_LOC155060,M_RNU12-2P,M_SSX9,M_CXORF67,M_EFCAB8,M_SRP14P1,M_LOC391343,...,M_ZWILCH,M_ZWINT,M_ZXDA,M_ZXDB,M_ZXDC,M_ZYG11A,M_ZYG11B,M_ZYX,M_ZZEF1,M_ZZZ3
0,2.0,-0.0361,NaN,0.3014,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-0.3588,-0.6952,-0.6728,-1.3532,-0.0299,-0.5507,-0.9856,-0.3259,0.5299,-1.6666
3,2.0,-0.1102,NaN,-0.1719,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-0.1447,-0.4123,-0.8120,-0.1139,-0.4753,-0.2675,-1.3015,0.2438,-0.7655,0.9903
4,2.0,-0.7052,NaN,0.3580,NaN,-0.0491,NaN,NaN,NaN,NaN,...,1.5699,0.5423,-1.7109,-0.6793,-0.5877,0.7986,-0.9206,-0.3501,-1.3085,-0.4572
5,2.0,1.4828,NaN,-0.0582,NaN,-0.0491,NaN,NaN,NaN,NaN,...,2.9688,0.1896,0.2491,0.4027,-0.2384,0.9772,0.0215,-1.3029,-1.0051,1.6254
6,2.0,-0.1719,NaN,0.1259,NaN,-0.0491,NaN,NaN,NaN,NaN,...,0.7023,0.3496,0.2206,0.9659,1.0893,-0.7623,1.0353,-0.4055,0.2360,0.6438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,1.0,-0.5763,NaN,-0.5543,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-0.4111,-0.3825,-0.2148,-0.6769,-0.1111,0.4151,-0.3884,1.3184,-0.3554,-0.2667
482,1.0,0.5183,NaN,-0.0826,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-0.9603,-0.1455,-0.0074,-0.2516,-0.3009,0.7224,-0.4032,0.2512,0.1328,-0.0146
484,1.0,0.6768,NaN,10.6747,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-1.1159,-0.3622,0.1483,-0.2998,0.8452,0.0677,-1.1697,-0.9595,-0.2514,-1.4445
485,1.0,-0.2252,NaN,-0.0288,NaN,-0.0491,NaN,NaN,NaN,NaN,...,-0.7841,-0.6561,0.1232,0.2636,1.9797,-0.1537,0.2651,1.1690,0.4163,-1.1685


#### 1. Omit variables that all of their values were zero. 

In [146]:
columns = data.columns
for element in columns:
    if (data[element] == 0).all():
        # drop column B if all the values are 0
        data = data.drop(element, axis=1)

#### 2. The NA values were imputed by the mean imputation method.data.mean()

In [147]:
# drop columns with all NaN values
data = data.dropna(axis=1, how='all')
# fill missing values with the mean of each column
data = data.fillna(data.mean())
data

,PATH_N_STAGE,M_UBE2Q2P2,M_LOC155060,M_SSX9,M_SDR16C6P,M_GTPBP6,M_EFCAB12,M_A1BG,M_A1CF,M_RBFOX1,...,M_ZWILCH,M_ZWINT,M_ZXDA,M_ZXDB,M_ZXDC,M_ZYG11A,M_ZYG11B,M_ZYX,M_ZZEF1,M_ZZZ3
0,2.0,-0.0361,0.3014,-0.0491,-0.1927,-0.2867,0.9115,0.2252,-0.1838,0.3830,...,-0.3588,-0.6952,-0.6728,-1.3532,-0.0299,-0.5507,-0.9856,-0.3259,0.5299,-1.6666
3,2.0,-0.1102,-0.1719,-0.0491,-0.1927,-0.2612,0.2185,-0.6294,-0.1838,-0.2837,...,-0.1447,-0.4123,-0.8120,-0.1139,-0.4753,-0.2675,-1.3015,0.2438,-0.7655,0.9903
4,2.0,-0.7052,0.3580,-0.0491,-0.1927,0.9095,-0.1748,-0.1893,-0.1838,-0.3850,...,1.5699,0.5423,-1.7109,-0.6793,-0.5877,0.7986,-0.9206,-0.3501,-1.3085,-0.4572
5,2.0,1.4828,-0.0582,-0.0491,-0.1927,-0.0856,-0.2188,-0.8000,-0.1838,-0.4006,...,2.9688,0.1896,0.2491,0.4027,-0.2384,0.9772,0.0215,-1.3029,-1.0051,1.6254
6,2.0,-0.1719,0.1259,-0.0491,0.6886,-0.1041,-0.9601,-0.4640,-0.1838,-0.3244,...,0.7023,0.3496,0.2206,0.9659,1.0893,-0.7623,1.0353,-0.4055,0.2360,0.6438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,1.0,-0.5763,-0.5543,-0.0491,-0.1927,-0.2603,-0.7611,-0.1714,-0.1838,-0.0231,...,-0.4111,-0.3825,-0.2148,-0.6769,-0.1111,0.4151,-0.3884,1.3184,-0.3554,-0.2667
482,1.0,0.5183,-0.0826,-0.0491,-0.1927,-0.2921,-0.8027,-0.3323,-0.1602,-0.1245,...,-0.9603,-0.1455,-0.0074,-0.2516,-0.3009,0.7224,-0.4032,0.2512,0.1328,-0.0146
484,1.0,0.6768,10.6747,-0.0491,-0.1927,-0.9893,0.0690,-0.3671,-0.1838,-0.3078,...,-1.1159,-0.3622,0.1483,-0.2998,0.8452,0.0677,-1.1697,-0.9595,-0.2514,-1.4445
485,1.0,-0.2252,-0.0288,-0.0491,-0.1927,-0.1738,-1.3305,0.1966,-0.1838,0.0035,...,-0.7841,-0.6561,0.1232,0.2636,1.9797,-0.1537,0.2651,1.1690,0.4163,-1.1685


In [148]:
data.reset_index(drop = True)

,PATH_N_STAGE,M_UBE2Q2P2,M_LOC155060,M_SSX9,M_SDR16C6P,M_GTPBP6,M_EFCAB12,M_A1BG,M_A1CF,M_RBFOX1,...,M_ZWILCH,M_ZWINT,M_ZXDA,M_ZXDB,M_ZXDC,M_ZYG11A,M_ZYG11B,M_ZYX,M_ZZEF1,M_ZZZ3
0,2.0,-0.0361,0.3014,-0.0491,-0.1927,-0.2867,0.9115,0.2252,-0.1838,0.3830,...,-0.3588,-0.6952,-0.6728,-1.3532,-0.0299,-0.5507,-0.9856,-0.3259,0.5299,-1.6666
1,2.0,-0.1102,-0.1719,-0.0491,-0.1927,-0.2612,0.2185,-0.6294,-0.1838,-0.2837,...,-0.1447,-0.4123,-0.8120,-0.1139,-0.4753,-0.2675,-1.3015,0.2438,-0.7655,0.9903
2,2.0,-0.7052,0.3580,-0.0491,-0.1927,0.9095,-0.1748,-0.1893,-0.1838,-0.3850,...,1.5699,0.5423,-1.7109,-0.6793,-0.5877,0.7986,-0.9206,-0.3501,-1.3085,-0.4572
3,2.0,1.4828,-0.0582,-0.0491,-0.1927,-0.0856,-0.2188,-0.8000,-0.1838,-0.4006,...,2.9688,0.1896,0.2491,0.4027,-0.2384,0.9772,0.0215,-1.3029,-1.0051,1.6254
4,2.0,-0.1719,0.1259,-0.0491,0.6886,-0.1041,-0.9601,-0.4640,-0.1838,-0.3244,...,0.7023,0.3496,0.2206,0.9659,1.0893,-0.7623,1.0353,-0.4055,0.2360,0.6438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,1.0,-0.5763,-0.5543,-0.0491,-0.1927,-0.2603,-0.7611,-0.1714,-0.1838,-0.0231,...,-0.4111,-0.3825,-0.2148,-0.6769,-0.1111,0.4151,-0.3884,1.3184,-0.3554,-0.2667
413,1.0,0.5183,-0.0826,-0.0491,-0.1927,-0.2921,-0.8027,-0.3323,-0.1602,-0.1245,...,-0.9603,-0.1455,-0.0074,-0.2516,-0.3009,0.7224,-0.4032,0.2512,0.1328,-0.0146
414,1.0,0.6768,10.6747,-0.0491,-0.1927,-0.9893,0.0690,-0.3671,-0.1838,-0.3078,...,-1.1159,-0.3622,0.1483,-0.2998,0.8452,0.0677,-1.1697,-0.9595,-0.2514,-1.4445
415,1.0,-0.2252,-0.0288,-0.0491,-0.1927,-0.1738,-1.3305,0.1966,-0.1838,0.0035,...,-0.7841,-0.6561,0.1232,0.2636,1.9797,-0.1537,0.2651,1.1690,0.4163,-1.1685


#### 3. The gene expressions with zero variance were removed. 

In [149]:
print('the shape of the data before removal of zero variance is', data.shape)
#find and remove zero variance
data = data.drop(data.var(axis = 0)[data.var(axis = 0) == 0].index.values, axis=1)
print('the shape of the data after removal of zero variance is', data.shape)

the shape of the data before removal of zero variance is (417, 19966)
the shape of the data after removal of zero variance is (417, 19966)


Therefore, there is no gene expression that has strictly zero variance. We suspected that we should set a threshold for zero variance in order to remove more features.

**According to the paper: after the variables with zero variance were removed, the number of variables was reduced to 4000. They had the greatest ANOVA F-values.**

Therefore, we calculated the ANOVA F-values and find the possible threshold. This is reasonable because if a feature with a high variance between groups and a low variance within groups, then it tends to have a higher ANOVA F-value.data[:,1:]

In [189]:
f_values, p_values = f_classif(data.iloc[:,1:],data['PATH_N_STAGE'])
f_pd = pd.DataFrame(f_values, columns = ['F-value'])

In [190]:
#Possible threshold 
threshold = 4.9821
count1 = (f_pd['F-value']>threshold).sum()
print('A threshold of', threshold, 'limited the numbers of features to',count1)

A threshold of 4.9821 limited the numbers of features to 4000


In [191]:
# create a new dataframe with the feature names and F-values
fnf = pd.DataFrame({'feature': data.iloc[:,1:].columns, 'F-value': f_values})

# sort the dataframe in descending order based on the F-values
sorted_fnf = fnf.sort_values('F-value', ascending=False)

#Selected the top 4000 features
columns = sorted_fnf[:4000]['feature']
selected = data[columns]
selected

,M_CTNNAL1,M_C16ORF91,M_PRSS27,M_HIST1H3J,M_OAS3,M_OAS1,M_CKS1B,M_MAP6D1,M_LOC100129935,M_GABPB2,...,M_HPDL,M_ZC3H11A,M_MAGEC3,M_TESPA1,M_RAB39A,M_ATG16L1,M_RNF135,M_CAPN14,M_RABL2A,M_ASB13
0,-0.7253,-0.0768,-0.0967,-0.6287,-0.4095,-0.1719,-0.5324,-0.0323,-0.0489,-0.5993,...,4.1173,0.3612,1.2856,-0.7000,-0.4913,-0.0141,-0.1810,-0.3636,0.0976,-1.3205
3,0.3567,-0.7217,-0.2737,0.0056,-0.7055,-0.6596,-0.7607,-0.7301,-0.0489,-0.7948,...,0.2793,0.1532,-0.4487,-0.5564,-0.2891,-0.4861,-0.0159,0.8203,0.4123,0.3274
4,-1.6328,-1.2149,-0.6054,0.3899,-0.3921,-0.1846,-0.8369,-0.5067,-0.0489,-2.0156,...,0.8328,-1.1808,-0.4487,-0.6206,0.0872,0.1618,1.3626,-0.1659,0.8552,0.2087
5,0.5051,0.2343,-0.5626,0.9624,-0.5634,-0.5755,-0.2063,0.7426,-0.0489,0.8774,...,-0.4139,0.7234,-0.4487,-0.6513,-0.1241,0.8600,-0.1750,-0.6490,-1.0373,0.2868
6,1.6196,0.3325,-0.4805,-0.6287,-0.2177,-0.5363,0.0658,-0.6350,-0.0489,0.4251,...,-0.2538,0.5924,-0.4487,-0.4398,-0.4335,-1.9213,-0.3402,-0.7131,-0.6063,0.4258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,-0.0506,-0.2335,0.3169,-0.0826,-0.1691,-0.2698,0.8429,0.2255,-0.0489,3.2326,...,0.3750,-0.5360,-0.4487,3.5292,-0.2015,-0.6994,0.9662,-0.0530,0.1612,-1.1149
482,0.5306,0.0369,-0.5095,0.5126,-0.2844,-0.4321,0.0187,0.6411,-0.0489,1.1949,...,-0.0431,0.2345,-0.4487,0.9109,0.0642,0.7365,0.3810,-0.3108,1.0893,0.5753
484,3.3271,1.7487,-0.3745,0.5228,0.0166,-0.1579,1.6761,4.1996,-0.0489,4.4300,...,-0.3415,1.4632,-0.4487,1.1339,0.3670,-0.3613,-0.6580,0.6786,1.1767,-0.6225
485,-1.1593,0.1446,4.7902,0.0459,3.2821,1.8775,0.7112,-0.4126,-0.0489,0.9277,...,-0.4544,1.8294,-0.4487,4.3256,0.0244,-0.2576,1.2125,0.5778,1.2353,-0.6806


Note the our table is sorted by ANOVA-F value

In [192]:
selected.insert(loc=0, column='PATH_N_STAGE', value=PATH_N_STAGE)
data = selected

In [197]:
data

,PATH_N_STAGE,M_CTNNAL1,M_C16ORF91,M_PRSS27,M_HIST1H3J,M_OAS3,M_OAS1,M_CKS1B,M_MAP6D1,M_LOC100129935,...,M_HPDL,M_ZC3H11A,M_MAGEC3,M_TESPA1,M_RAB39A,M_ATG16L1,M_RNF135,M_CAPN14,M_RABL2A,M_ASB13
0,2.0,-0.7253,-0.0768,-0.0967,-0.6287,-0.4095,-0.1719,-0.5324,-0.0323,-0.0489,...,4.1173,0.3612,1.2856,-0.7000,-0.4913,-0.0141,-0.1810,-0.3636,0.0976,-1.3205
3,2.0,0.3567,-0.7217,-0.2737,0.0056,-0.7055,-0.6596,-0.7607,-0.7301,-0.0489,...,0.2793,0.1532,-0.4487,-0.5564,-0.2891,-0.4861,-0.0159,0.8203,0.4123,0.3274
4,2.0,-1.6328,-1.2149,-0.6054,0.3899,-0.3921,-0.1846,-0.8369,-0.5067,-0.0489,...,0.8328,-1.1808,-0.4487,-0.6206,0.0872,0.1618,1.3626,-0.1659,0.8552,0.2087
5,2.0,0.5051,0.2343,-0.5626,0.9624,-0.5634,-0.5755,-0.2063,0.7426,-0.0489,...,-0.4139,0.7234,-0.4487,-0.6513,-0.1241,0.8600,-0.1750,-0.6490,-1.0373,0.2868
6,2.0,1.6196,0.3325,-0.4805,-0.6287,-0.2177,-0.5363,0.0658,-0.6350,-0.0489,...,-0.2538,0.5924,-0.4487,-0.4398,-0.4335,-1.9213,-0.3402,-0.7131,-0.6063,0.4258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,1.0,-0.0506,-0.2335,0.3169,-0.0826,-0.1691,-0.2698,0.8429,0.2255,-0.0489,...,0.3750,-0.5360,-0.4487,3.5292,-0.2015,-0.6994,0.9662,-0.0530,0.1612,-1.1149
482,1.0,0.5306,0.0369,-0.5095,0.5126,-0.2844,-0.4321,0.0187,0.6411,-0.0489,...,-0.0431,0.2345,-0.4487,0.9109,0.0642,0.7365,0.3810,-0.3108,1.0893,0.5753
484,1.0,3.3271,1.7487,-0.3745,0.5228,0.0166,-0.1579,1.6761,4.1996,-0.0489,...,-0.3415,1.4632,-0.4487,1.1339,0.3670,-0.3613,-0.6580,0.6786,1.1767,-0.6225
485,1.0,-1.1593,0.1446,4.7902,0.0459,3.2821,1.8775,0.7112,-0.4126,-0.0489,...,-0.4544,1.8294,-0.4487,4.3256,0.0244,-0.2576,1.2125,0.5778,1.2353,-0.6806


#### 4. The gene expression with great correlation with the other genes (threshold of 0.9) were removed.

According to the paper, at this step, the number of the variables was 3882.

In [198]:
#make correlation matrix using np
trans_matrix = np.array(data.iloc[:, 1:]).transpose()
corr_use = np.corrcoef(trans_matrix)
trans_matrix = 0
print(corr_use)

[[ 1.          0.35620461  0.14730211 ...  0.01141118  0.01872491
  -0.04400723]
 [ 0.35620461  1.          0.36580031 ...  0.14014198  0.23203951
   0.24337942]
 [ 0.14730211  0.36580031  1.         ... -0.05419722  0.19683402
   0.06414602]
 ...
 [ 0.01141118  0.14014198 -0.05419722 ...  1.          0.05324134
   0.03268104]
 [ 0.01872491  0.23203951  0.19683402 ...  0.05324134  1.
   0.06302614]
 [-0.04400723  0.24337942  0.06414602 ...  0.03268104  0.06302614
   1.        ]]


In [199]:
#replace the diagnal value with 0
np.fill_diagonal(corr_use, 0)
#replace the upper triangle with 0s
corr_use = np.triu(corr_use)
print(corr_use)

[[ 0.          0.35620461  0.14730211 ...  0.01141118  0.01872491
  -0.04400723]
 [ 0.          0.          0.36580031 ...  0.14014198  0.23203951
   0.24337942]
 [ 0.          0.          0.         ... -0.05419722  0.19683402
   0.06414602]
 ...
 [ 0.          0.          0.         ...  0.          0.05324134
   0.03268104]
 [ 0.          0.          0.         ...  0.          0.
   0.06302614]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [219]:
#drop those that exceeds thershold
threshold = 0.9
ind = np.argwhere(corr_use > threshold)
data = data.drop(data.columns[ind[:,1]], axis = 1)
data.to_csv('data_processed.csv', index = False)

In [220]:
data

,PATH_N_STAGE,M_CTNNAL1,M_C16ORF91,M_PRSS27,M_HIST1H3J,M_OAS1,M_CKS1B,M_MAP6D1,M_LOC100129935,M_GABPB2,...,M_HLA-DPB1,M_HPDL,M_ZC3H11A,M_TESPA1,M_RAB39A,M_ATG16L1,M_RNF135,M_CAPN14,M_RABL2A,M_ASB13
0,2.0,-0.7253,-0.0768,-0.0967,-0.6287,-0.1719,-0.5324,-0.0323,-0.0489,-0.5993,...,-0.7532,4.1173,0.3612,-0.7000,-0.4913,-0.0141,-0.1810,-0.3636,0.0976,-1.3205
3,2.0,0.3567,-0.7217,-0.2737,0.0056,-0.6596,-0.7607,-0.7301,-0.0489,-0.7948,...,-0.8281,0.2793,0.1532,-0.5564,-0.2891,-0.4861,-0.0159,0.8203,0.4123,0.3274
4,2.0,-1.6328,-1.2149,-0.6054,0.3899,-0.1846,-0.8369,-0.5067,-0.0489,-2.0156,...,-1.1604,0.8328,-1.1808,-0.6206,0.0872,0.1618,1.3626,-0.1659,0.8552,0.2087
5,2.0,0.5051,0.2343,-0.5626,0.9624,-0.5755,-0.2063,0.7426,-0.0489,0.8774,...,-0.6370,-0.4139,0.7234,-0.6513,-0.1241,0.8600,-0.1750,-0.6490,-1.0373,0.2868
6,2.0,1.6196,0.3325,-0.4805,-0.6287,-0.5363,0.0658,-0.6350,-0.0489,0.4251,...,-0.5394,-0.2538,0.5924,-0.4398,-0.4335,-1.9213,-0.3402,-0.7131,-0.6063,0.4258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,1.0,-0.0506,-0.2335,0.3169,-0.0826,-0.2698,0.8429,0.2255,-0.0489,3.2326,...,2.2672,0.3750,-0.5360,3.5292,-0.2015,-0.6994,0.9662,-0.0530,0.1612,-1.1149
482,1.0,0.5306,0.0369,-0.5095,0.5126,-0.4321,0.0187,0.6411,-0.0489,1.1949,...,0.6081,-0.0431,0.2345,0.9109,0.0642,0.7365,0.3810,-0.3108,1.0893,0.5753
484,1.0,3.3271,1.7487,-0.3745,0.5228,-0.1579,1.6761,4.1996,-0.0489,4.4300,...,1.0021,-0.3415,1.4632,1.1339,0.3670,-0.3613,-0.6580,0.6786,1.1767,-0.6225
485,1.0,-1.1593,0.1446,4.7902,0.0459,1.8775,0.7112,-0.4126,-0.0489,0.9277,...,3.6217,-0.4544,1.8294,4.3256,0.0244,-0.2576,1.2125,0.5778,1.2353,-0.6806


We have less variables than the paper.

### Verify whether the desired genes were included in our table

1. by the known list of features

In [224]:
tfeatures = ['M_SPAG1', 'M_PLEKHF2','M_RALGPS1','M_OMP', 'M_CTNNAL1','M_ZBTB34','M_ODF2','M_FKBP15','M_GOLGA1','M_CHAF1B',
                'M_ORAOV1','M_ZNF174','M_C20ORF204','M_ZNF263','M_HIST1H2BI','M_NUP85','M_KIF18A','M_ARPC5L',
             'M_ANKRD13D','M_ZBTB43','M_ABCA17P']
ins = []
for feature in tfeatures:
    if feature in data:
      ins.append(data[feature])
ins = pd.DataFrame(ins).transpose()
ins.shape

(417, 21)

2. by the Average Standardized log2 level (N1)

In [225]:
N1 = data[data['PATH_N_STAGE'] == 1.0]
N0 = data[data['PATH_N_STAGE'] == 2.0]

In [231]:
meansN1 = []
meansN0 = []
for feature in tfeatures:
    meansN1.append(N1[feature].mean())
    meansN0.append(N0[feature].mean())

In [236]:
df = {'Gene': tfeatures,
        'Average (N1)': meansN1,
        'Average (N0)': meansN0}

# convert the dictionary to a dataframe
table = pd.DataFrame(df)

In [237]:
table

,Gene,Average (N1),Average (N0)
0,M_SPAG1,1.475490,0.229783
1,M_PLEKHF2,1.298871,0.287055
2,M_RALGPS1,1.233461,0.121630
3,M_OMP,1.198101,0.020145
4,M_CTNNAL1,1.180699,0.074221
5,M_ZBTB34,1.089381,0.155228
6,M_ODF2,1.020451,0.078641
7,M_FKBP15,1.014252,0.117141
8,M_GOLGA1,1.000230,0.091729
9,M_CHAF1B,0.874740,0.004969
